In [1]:
import pandas as pd

dataset = pd.read_csv(r"movie_reviews.csv.bz2")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [2]:
dataset.isna().sum()

review       0
sentiment    0
dtype: int64

In [3]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
import contractions
def preprocess(text):
    soup = BeautifulSoup(text,'html.parser')
    text = soup.get_text()
    words = word_tokenize(text)
    text = " ".join([word.lower() for word in words if word not in stop])
    text = re.sub(r'[^a-zA-Z0-9\s]',"",text,re.I)
    text = text.strip()
    text = contractions.fix(text)
    return text

In [4]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
from tqdm import tqdm
tqdm.pandas()

In [6]:
4+4

8

In [7]:
dataset['review'] = dataset['review'].progress_apply(preprocess)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [03:34<00:00, 233.25it/s]


In [9]:
reviews = dataset['review'].values
sentiment= dataset['sentiment'].values

In [10]:
train_Reviews = reviews[:35000]
train_sentiments = reviews[:35000]

In [11]:
test_reviews = reviews[35000:]
test_sentiments = reviews[35000:]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
#Bag of words
cv = CountVectorizer(binary = False,min_df = 5,max_df=1.0,ngram_range= (1,2))

In [15]:
cv_train_features = cv.fit_transform(train_Reviews)

In [16]:
cv_test_features = cv.transform(test_reviews)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
lr = LogisticRegression(penalty = 'l2',max_iter = 500,C = 1,solver = 'lbfgs')

In [19]:
lr.fit(cv_train_features,train_sentiments)

c:\users\aadhi\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


MemoryError: 

In [ ]:
lr_predictions = lr.predict(cv_test_features)

In [ ]:
labels = ['negative','positive']
print(classification_report(test_sentiments,lr_predictions))

In [ ]:
pd.DataFrame(confusion_matrix(test_sentiments,lr_predictions),index = labels,columns = labels)